In [ ]:
%pip install m3u8
%pip install pycryptodome
%pip install python-slugify==8.0.1
%pip uninstall crypto 
%pip uninstall pycrypto
%pip install pycryptodomex
%pip install pycryptodome


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.



In [1]:
import m3u8
import requests
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES
import requests
from tqdm import tqdm
import binascii
from http.cookies import SimpleCookie
from bs4 import BeautifulSoup
import re
from slugify import slugify

In [2]:

def decrypt_video(encrypted_data, key, iv):
    """
    as a .ts file is encrypted with AES-128 (CBC) algorithm, decrypt it using pycryptodome
    :param encrypted_data: .ts file data
    :param key: fetched .key file
    :param iv: sequence number (iv is not specified)
    :return: decrypted segment
    """
    encrypted_data = pad(data_to_pad=encrypted_data, block_size=AES.block_size)
    aes = AES.new(key=key, mode=AES.MODE_CBC, iv=iv)
    decrypted_data = aes.decrypt(encrypted_data)
    return decrypted_data

def binify(x):
    """
    convert sequence number(int) to 128bit representation for AES-128 decryption
    """
    h = hex(x)[2:].rstrip('L')
    return binascii.unhexlify('0'*(32-len(h))+h)


def get_soup(lecture_url, cookie_str):
    cookie = SimpleCookie()
    cookie.load(cookie_str)
    cookies = {k: v.value for k, v in cookie.items()}
    resp = requests.get(lecture_url, cookies=cookies)
    soup = BeautifulSoup(resp.text, 'html.parser') if resp is not None else None
    return soup

def download_video(uri, file_name, encrypted=True):
    playlist = m3u8.load(uri=uri)
    seq_len = len(playlist.segments)
    if encrypted:
        key = requests.get(playlist.keys[-1].absolute_uri).content
    for i in tqdm(range(seq_len)):
        seg = playlist.segments[i]
        data = requests.get(seg.absolute_uri).content
        iv = binify(i+1)
        if encrypted:
            data = decrypt_video(data, key, iv)
        with open(file_name,"ab" if i != 0 else "wb") as f:
            f.write(data)

def get_popup_list(lecture_soup, cookie_str, popup_type='vod'):
    popup_list = []
    for li in tqdm(lecture_soup.find_all("li", {"class": popup_type})):
        a = li.find('a')
        video_name = a.find("span", {"class":"instancename"}).text
        viewer_url = re.search("window.open\(\'(.+?)\',", a['onclick']).group(1)
        if popup_type == 'vod':
            vod_soup = get_soup(viewer_url, cookie_str)
            if vod_soup is not None:
                source = vod_soup.find("source")
                if source is not None:
                    video_url = source["src"]
                    popup_list.append({'name': video_name, 'url': video_url, 'type': popup_type})
        elif popup_type == 'laby':
            viewer_url = 'https://ys.learnus.org' + viewer_url
            popup_soup = get_soup(viewer_url, cookie_str)
            content_id = re.search("content_id=(.+?)&", str(popup_soup)).group(1)
            content_url = "https://alrs.yonsei.ac.kr/HttpAPI/GetContentinfobyPlayer?cnt_uid={content_id}&cos_uid=&cnt_playtype=0?content_id={content_id}".format(content_id=content_id)
            content_soup = get_soup(content_url, cookie_str)
            content_url1 = content_soup.find("content_url1").text
            content_url2 = content_soup.find("content_url2").text
#             print(content_url1, content_url2)
            popup_list.append({'name': video_name + '_1', 'url': content_url1, 'type': popup_type})
            popup_list.append({'name': video_name + '_2', 'url': content_url2, 'type': popup_type})
    return popup_list


In [ ]:
cookie_str = input ("Enter cookie string:") 
lecture_url = input("Enter Lecture URL : ") 
# cookie_str = 'login_check=true; _gcl_au=1.1.1450246283.1677722439; au_id=254dbcf42af70e0817e5981417f2bc828e5-613c; _gid=GA1.2.1322644791.1678774690; MoodleSession=f3ih88onre12vipbi0v5c8vji7; login_check=true; ubboard_read=O%CE%0F%03a%11%93%81%A6%D7%15%A4Q%0D%7F%3B%DCpnw%A9%88%8C%C7%C7%B3%11%D0f%5E%0E%87i%C2%80i%03%CFb%7D%CF%DD%EC%A8; _ga_M9Z4NCPPY6=GS1.1.1678863189.287.1.1678863955.0.0.0; _ga=GA1.2.358086843.1645707131; _gat_UA-160620709-35=1; hide_menu=/mod/quiz,/mod/assign,/mod/vod,/mod/econtents'
# lecture_url = 'https://ys.learnus.org/course/view.php?id=229587'
lecture_soup = get_soup(lecture_url, cookie_str)
if lecture_soup is not None:
    vod_list = get_popup_list(lecture_soup, cookie_str, 'vod')
    laby_list = get_popup_list(lecture_soup, cookie_str, 'laby')

video_list = vod_list + laby_list

In [ ]:
import ipywidgets as widgets
from IPython.display import display
layout = widgets.Layout(width='auto', height='auto') #set width and height
output = widgets.Output()
list_checks = []
for item in video_list:
    url = item['url']
    name = item['name']
    checkbox = widgets.Checkbox(value=True, description=name, indent=False, display='flex', layout=layout)
    list_checks.append(checkbox)
    display(checkbox, output)

button = widgets.Button(description="Download")
def on_button_clicked(b):
  # Display the message within the output widget.
    for i, check in enumerate(list_checks):
        _video = video_list[i]
        url, name, _type, checked = _video['url'], _video['name'], _video['type'], check.value
        name = slugify(name, allow_unicode=True, separator='_') + '.mp4'
        if checked:
            if _type == 'vod':
                download_video(url, name, encrypted=True)
            elif _type == 'laby':
                download_video(url, name, encrypted=False)
button.on_click(on_button_clicked)
display(button, output)